In [65]:
# installations
!pip install itertools
!pip install random
!pip install copy

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement copy (from versions: none)
ERROR: No matching distribution found for copy

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [51]:
#Update your token
STUDENT_TOKEN = 'GABRIEL DE OLAGUIBEL'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

## Kakuro Implementation:
- Using Backtracking
- Using constraint propagation
- Using MRV heuristic
- Without recursion (Stack based)
- Prints amount of backtracks


In [52]:
# Some helper functions for printing the board
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [53]:
# Board representation
class Kakuro:
    def __init__(self, puzzle):
        self.board = puzzle['board']
        self.h_constraints = puzzle['h_constraints']
        self.v_constraints = puzzle['v_constraints']
        self.id = puzzle.get('id', None)

    
    # Valilidation Methods:

    def check_horizontal(self, row, col): # this function will check the sum of the horizontal sequence
        target_sum = self.h_constraints[row-1][0]
        total, count = 0, 0
        for c in range(col, len(self.board[row])): 
            cell = self.board[row][c]
            if cell == 'X':  # means it the end of th sequence
                break
            if cell == '.':  # find empty cell
                continue
            total += int(cell) # suming the values
            count += 1
        if count == 0:  # cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded so it is not valid
        if 'X' not in self.board[row][col:] and total != target_sum: # checking if it is not the target sum
            return False  
        return True

    # follows a similar approach to the horizontal check
    def check_vertical(self, row, col):
        target_sum = self.v_constraints[col-1][0] 
        total, count = 0, 0
        for r in range(row, len(self.board)):
            cell = self.board[r][col]
            if cell == 'X':  
                break
            if cell == '.':  
                continue
            total += int(cell)
            count += 1
        if count == 0:  
            return True
        if total > target_sum:
            return False  
        if 'X' not in [self.board[r][col] for r in range(row, len(self.board))] and total != target_sum:
            return False  
        return True

    def is_valid(self):
        # Check horizontal constraints
        for r, row in enumerate(self.board): 
            for c, value in enumerate(row):
                if value != 'X' and value != '.': # i'll Check only cells with values
                    # Check the sum of horizontal sequence using the function
                    if not self.check_horizontal(r, c): 
                        return False
                    # now checking vertical
                    if not self.check_vertical(r, c):
                        return False
        return True
    




### Constraint propagation

In [54]:
import itertools # using this library to generate all possible combinations of a list which is useful when applying constraints


# function to Determine the possible values for each empty cell in a sequence by considering all other cells.
def generate_possible_values(empty_indices, filled_cells, target_sum): 

    remaining_sum = target_sum - sum(filled_cells)
    
    # Exclud numbers already used in the filled cells when generating combinations
    available_numbers = [num for num in range(1, 10) if num not in filled_cells]
    
    # Generate all possible combinations of numbers that can fill the empty cells and match the remaining sum.
    valid_permutations = [seq for seq in itertools.permutations(available_numbers, len(empty_indices)) 
                          if sum(seq) == remaining_sum and all(num not in filled_cells for num in seq)] # checking if the numbers are not in the filled cells
    
    # For each cell position, determine the possible values it can take
    possible_values_per_cell = [set() for _ in empty_indices] 
    for perm in valid_permutations:
        for i, val in enumerate(perm): # i is index and val is obviously the value btw
            possible_values_per_cell[i].add(val)
    
    return possible_values_per_cell 

# function provides a sequence (horizontal or vertical) that the cell belongs to and its sum constraint
def find_sequence_and_sum(board, row, col, constraints, is_horizontal):
    if is_horizontal:
        start, end = col, col 
        while start > 0 and board[row][start - 1] != 'X': # checking if the cell were looking at is not the first one in the sequence
            start -= 1 # move left to find it
        while end < len(board[row]) and board[row][end] != 'X': # checking if the cell were looking at is not the last one in the sequence
            end += 1 # move right to find it
        sequence = board[row][start:end] 
        target_sum = constraints[row - 1][0] if row > 0 else None # if the row is 0 then there is no constraint
    else: # same thing but for vertical:
        start, end = row, row
        while start > 0 and board[start - 1][col] != 'X':
            start -= 1
        while end < len(board) and board[end][col] != 'X':
            end += 1
        sequence = [board[i][col] for i in range(start, end)]
        target_sum = constraints[col - 1][0] if col > 0 else None # if the col is 0 then there is no constraint

    return sequence, target_sum

# function updates the board with the possible values for each sequence given the filled cells and the sum constraint
def propagate_constraints(board, row, col, h_constraints, v_constraints): 
    # Find the sequemces affected by the placement of a number 
    h_sequence, h_target_sum = find_sequence_and_sum(board, row, col, h_constraints, True)
    v_sequence, v_target_sum = find_sequence_and_sum(board, row, col, v_constraints, False)

    # Initialize possibilities for the horizontal and vertical sequences
    # if the cell is empty then it can be any number from 1 to 9 initially
    h_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in h_sequence]
    v_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in v_sequence]

    # use generate_possible_values for the empty cells in the horizontal and vertical sequences
    if h_target_sum: # if there is a constraint
        h_empty_indices = [i for i, x in enumerate(h_sequence) if x == '.'] # geting the index of the empty cells
        h_filled_cells = [int(x) for x in h_sequence if x != '.'] # getting the filled cells
        h_possibilities_for_empty = generate_possible_values(h_empty_indices, h_filled_cells, h_target_sum) # calling the function
        for i, index in enumerate(h_empty_indices):
            h_possibilities[index] = h_possibilities_for_empty[i]

    if v_target_sum:
        v_empty_indices = [i for i, x in enumerate(v_sequence) if x == '.']
        v_filled_cells = [int(x) for x in v_sequence if x != '.']
        v_possibilities_for_empty = generate_possible_values(v_empty_indices, v_filled_cells, v_target_sum)
        for i, index in enumerate(v_empty_indices):
            v_possibilities[index] = v_possibilities_for_empty[i]

    # Get the index of the current cell in the horizontal and vertical sequences
    h_index = h_sequence.index('.') if '.' in h_sequence else None
    v_index = v_sequence.index('.') if '.' in v_sequence else None

    # Combine the possibilities for the current cell from the horizontal and vertical sequences
    combined_possibilities = set(range(1, 10)) # intital possibilities
    if h_index is not None and h_index < len(h_possibilities): 
        combined_possibilities.intersection_update(h_possibilities[h_index]) # using intersection_update which updates the set with the intersection of itself and another 
    if v_index is not None and v_index < len(v_possibilities):
        combined_possibilities.intersection_update(v_possibilities[v_index]) 

    # Return the combined possibilities for the current sequence
    return combined_possibilities 

# this function returns the possible values for a cell by combining the constraints from the horizontal and vertical sequences
def get_possible_values(board, row, col, h_constraints, v_constraints):
    possible_values = set(range(1, 10)) # intital possibilities
    # Propagate constraints from the current board state
    combined_possibilities = propagate_constraints(board, row, col, h_constraints, v_constraints)
    # The combined possibilities are for the current cell only
    possible_values.intersection_update(combined_possibilities)
    return possible_values # this is different from the one in the previous function because this one is for a single

# Testing/Debug get_possible_values to ensure its identifying the sequences and the possible values correctly
'''
problem = {'board': 
           [['X', 'X', 'X', 'X', 'X'],
           ['X', 'X', 'X', '.', '.'],
           ['X', '.', '.', '.', '.'],
           ['X', '.', '.', '.', '.'],
           ['X', '.', '.', 'X', 'X']],
 'h_constraints': [[5], [13], [14], [11]],
 'id': 2,
 'v_constraints': [[23], [7], [6], [7]]}

kakuro = Kakuro(problem)
print(get_possible_values(kakuro.board, 1, 4, kakuro.h_constraints, kakuro.v_constraints))

h_sequence, h_target_sum = find_sequence_and_sum(problem['board'], 1, 4, problem['h_constraints'], True)
v_sequence, v_target_sum = find_sequence_and_sum(problem['board'], 1, 4, problem['v_constraints'], False)
print("Horizontal word:", h_sequence)
print("Horizontal target sum:", h_target_sum) 

print("Vertical word:", v_sequence)
print("Vertical target sum:", v_target_sum) 
'''



'\nproblem = {\'board\': \n           [[\'X\', \'X\', \'X\', \'X\', \'X\'],\n           [\'X\', \'X\', \'X\', \'.\', \'.\'],\n           [\'X\', \'.\', \'.\', \'.\', \'.\'],\n           [\'X\', \'.\', \'.\', \'.\', \'.\'],\n           [\'X\', \'.\', \'.\', \'X\', \'X\']],\n \'h_constraints\': [[5], [13], [14], [11]],\n \'id\': 2,\n \'v_constraints\': [[23], [7], [6], [7]]}\n\nkakuro = Kakuro(problem)\nprint(get_possible_values(kakuro.board, 1, 4, kakuro.h_constraints, kakuro.v_constraints))\n\nh_sequence, h_target_sum = find_sequence_and_sum(problem[\'board\'], 1, 4, problem[\'h_constraints\'], True)\nv_sequence, v_target_sum = find_sequence_and_sum(problem[\'board\'], 1, 4, problem[\'v_constraints\'], False)\nprint("Horizontal word:", h_sequence)\nprint("Horizontal target sum:", h_target_sum) \n\nprint("Vertical word:", v_sequence)\nprint("Vertical target sum:", v_target_sum) \n'

### Minimum remaining values implementation

In [55]:
# heuristic function to determine the cell with the minimum remaining values for efficientcy

def calculate_mrv(board, h_constraints, v_constraints):
    mrv_cells = [] 
    # find all empty cells
    for row in range(len(board)):
        for col in range(len(board[0])):
            if board[row][col] == '.':
                # Get the possible values for each empty cell
                possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)
                mrv_cells.append(((row, col), len(possible_values))) # add it to the list with the length of the possible values
    # Sort by the number of possible values, which is the second item in the tuples
    mrv_cells.sort(key=lambda item: item[1])
    '''
    # used this to show the sorted MRV cells for each step to make sure its working
    print(f"MRV cells sorted by possible values: {mrv_cells}")
    '''
    return mrv_cells

## Solver implementation:
- ### Non-recursive

In [56]:
def solve_kakuro(board, h_constraints, v_constraints):

    backtracks = 0  

    # Initialize MRV with the first call to calculate_mrv.
    mrv_cells = calculate_mrv(board, h_constraints, v_constraints)
    if not mrv_cells: # edge case
        print("No empty cells found, puzzle is already solved or invalid.") # Used for debug 
        return board, backtracks

    # Start with the cell with the minimum remaining values (first in the list)
    (row, col), _ = mrv_cells[0] 
    #print(f"Starting with cell ({row}, {col}) with minimum remaining values.") # used this to check if it was working

    # Initialize the stack with the first cell
    stack = [(board, row, col)] 

    while stack:
        board, row, col = stack.pop()  # Get the current state and cell to fill
        '''
        print(f"\nCurrent state: {board}")
        print(f"Current position: ({row}, {col})")
        print(f"Stack size: {len(stack)}, Backtracks: {backtracks}") 
        '''
        # Get possible values for the current cell
        possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)

        # If there are no possible values then backtracking is needed
        if not possible_values:
            backtracks += 1
            # print("No possible values, backtracking...")
            continue  # Backtrack to the previous state. Continue here because the stack already has the previous state 

        # Try each possible value and push the new state onto the stack
        for value in possible_values:
            new_board = [list(r) for r in board]  # Create a copy of the board
            new_board[row][col] = str(value)  # Assign the value as a string
            #print(f"Trying value {value} at cell ({row}, {col})")

            # Update MRV after the assignment
            next_mrv_cells = calculate_mrv(new_board, h_constraints, v_constraints)
            if next_mrv_cells: 
                next_row, next_col = next_mrv_cells[0][0] # Get the next cell with minimum remaining values
                #print(f"Next cell to try is ({next_row}, {next_col}) with MRV.")
                stack.append((new_board, next_row, next_col))  # Push the new state with MRV cell
            else:
                # If MRV list is empty, it means all cells are filled and passed all constraints
                #print("Solution found")
                return new_board, backtracks

    # If the stack was empty, no solution was found
    print("No solution found.")
    return None, backtracks

### Test To Server

In [57]:
# Get the problem from the server
problem = get_problem('kakuro')

# print what it got from the server
pprint.pprint(problem)
print("\n")

# extract the board and constraints
board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

# print the initial board nicely
print('Input puzzle:')
pretty_print(board, h_const, v_const)
print("\n")

# Solve the puzzle 
solution, backtracks = solve_kakuro(board, h_const, v_const)
solution, backtracks

# print the solution nicely
print('\nSolution:')
pretty_print(solution, problem['h_constraints'], problem['v_constraints'])

answer = {
    'board': solution,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

submit_answer('kakuro', answer)

# print the number of backtracks
print(f"Backtracks: {backtracks}")

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', 'X', '.', '.']],
 'h_constraints': [[11], [14], [20], [17]],
 'id': 3,
 'v_constraints': [[16], [7], [23], [16]]}


Input puzzle:
      16  7 23 16
    X  X  X  X  X
11  X .. ..  X  X
14  X .. .. ..  X
20  X  X .. .. ..
17  X  X  X .. ..



Solution:
      16  7 23 16
    X  X  X  X  X
11  X  9  2  X  X
14  X  7  1  6  X
20  X  X  4  9  7
17  X  X  X  8  9
Ignore: not checking values yet
Backtracks: 0


# Sudoku Implementation:
- ### Using local search

In [58]:
# helper functins

import random

# Helper functions as provided from professor
def valid_row(sudoku_board, row): 
    assigned = []
    for value in sudoku_board[row]:
        if value != 0:
            if value in assigned:
                return False
            else:
                assigned.append(value)
    return True

def valid_column(sudoku_board, column): 
    assigned = []
    for row in sudoku_board:
        value = row[column]
        if value != 0:
            if value in assigned:
                return False
            else:
                assigned.append(value)
    return True

def valid_quadrant(sudoku_board, quadrant):
    starting_row = (quadrant // 3) * 3
    starting_column = (quadrant % 3) * 3
    assigned = []
    for row in sudoku_board[starting_row : starting_row + 3]:
        for value in row[starting_column : starting_column + 3]:
            if value != 0:
                if value in assigned:
                    return False
                assigned.append(value)
    return True

def complete_solution(sudoku_board):
    for row in sudoku_board:
        for value in row:
            if value == 0:
                return False
    return True

def valid_solution(sudoku_board): 
    for i in range(9):
        if not valid_row(sudoku_board, i):
            return False
        if not valid_column(sudoku_board, i):
            return False
        if not valid_quadrant(sudoku_board, i):
            return False
    return True

def check_solution(puzzle, solution):
    for row in range(9):
        for col in range(9):
            if puzzle[row][col] != 0 and puzzle[row][col] != solution[row][col]:
                return 'Solution is not valid, changes a fixed value in the puzzle'
    if not valid_solution(solution):
        return 'Solution does not respect the constraints'
    if not complete_solution(solution):
        return 'Solution is only partial'
    return 'Solution is valid'

def pretty_print(sudoku_board):
    for row in sudoku_board:
        print(" ".join(map(str, row)))

def sudoku_string_to_matrix(sudoku): 
    sudoku_board = []
    for i in range(9):
        starting_row = i * 9
        row = []
        for j in range(9):
            row.append(int(sudoku[starting_row + j]))
        sudoku_board.append(row)
    return sudoku_board

def sudoku_matrix_to_string(sudoku_board): 
    return ''.join(map(lambda row: ''.join(map(str, row)), sudoku_board))

def random_initialize_sudoku(puzzle): 
    sudoku_board = sudoku_string_to_matrix(puzzle)
    for i in range(9):
        for j in range(9):
            if sudoku_board[i][j] == 0:
                sudoku_board[i][j] = random.randint(1, 9)
    return sudoku_board

def count_conflicts(sudoku_board, row, col): 
    conflicts = 0
    value = sudoku_board[row][col]

    for i in range(9):
        if i != col and sudoku_board[row][i] == value: # checking the row for conflicts
            conflicts += 1

    for i in range(9):
        if i != row and sudoku_board[i][col] == value: # checking the column for conflicts
            conflicts += 1

    starting_row = (row // 3) * 3
    starting_column = (col // 3) * 3
    for i in range(3):
        for j in range(3):
            if (starting_row + i) != row and (starting_column + j) != col and sudoku_board[starting_row + i][starting_column + j] == value: # checking the quadrant for conflicts
                conflicts += 1

    return conflicts

'''
# Test the helper functions with a sample Sudoku string
sudoku_string = '000005000037800006200167800300009720905783060470200089890502600540900000000400593'
#sudoku_string2 = '003709008108346957976810004009063785380157692567080340890230570632578419005691023'
initialized_sudoku = random_initialize_sudoku(sudoku_string)
pretty_print(initialized_sudoku)
print("\nConflicts at (0, 0):", count_conflicts(initialized_sudoku, 0, 0))
print("\nCheck if the solution is valid:", check_solution(sudoku_string_to_matrix(sudoku_string), initialized_sudoku))
'''


'\n# Test the helper functions with a sample Sudoku string\nsudoku_string = \'000005000037800006200167800300009720905783060470200089890502600540900000000400593\'\n#sudoku_string2 = \'003709008108346957976810004009063785380157692567080340890230570632578419005691023\'\ninitialized_sudoku = random_initialize_sudoku(sudoku_string)\npretty_print(initialized_sudoku)\nprint("\nConflicts at (0, 0):", count_conflicts(initialized_sudoku, 0, 0))\nprint("\nCheck if the solution is valid:", check_solution(sudoku_string_to_matrix(sudoku_string), initialized_sudoku))\n'

In [63]:
# Local search 

from copy import deepcopy

def min_conflicts(board_string):

    # inializations
    board = sudoku_string_to_matrix(board_string)
    current_state = random_initialize_sudoku(board_string)

    # keep track of the cells that can be changed so I dont change the fixed ones
    changeable_cells = [(i, j) for i in range(9) for j in range(9) if board[i][j] == 0] 
    
    # Instead of doing max iterations, I will restart if there is no improvement after a certain number of iterations so it keeps trying to find a solution
    steps = 0 
    max_steps = 5000 # can be renamed to max_iterations

    while not valid_solution(current_state):
        conflicts = deepcopy(changeable_cells) # to avoid changing the original list which would be a mess
        
        for row, col in conflicts: 
            better_num = current_state[row][col] 
            min_conflicts = count_conflicts(current_state, row, col) # get the number of conflicts for the current_state cell
            
            # find the number that results in the fewest conflicts
            number_candidate = []
            for num in range(1, 10):
                temp = deepcopy(current_state) 
                temp[row][col] = num
               
                current_state_conflicts = count_conflicts(temp, row, col)
                if current_state_conflicts < min_conflicts:
                    number_candidate = [num] # add to list of number candidates since it has the fewer conflicts
                    min_conflicts = current_state_conflicts
                elif current_state_conflicts == min_conflicts: # if it has the same number of conflicts then add it to the list
                    number_candidate.append(num)
            # pick a random number from the list of candidates
            if number_candidate:
                better_num = number_candidate[random.randint(0, len(number_candidate) - 1)] # pick a random number from the list of candidates 
            # update the current state with the beter number
            current_state[row][col] = better_num  

            if min_conflicts < count_conflicts(current_state, row, col): # check if there is an improvement 
                steps = 0 # I will reset the steps if there is an improvement
                break
            else:
                steps += 1

        if steps > max_steps:  # restart if no improvement
            current_state = random_initialize_sudoku(board_string)
            steps = 0

    return current_state

In [64]:
# Test to server
problem = get_problem('easy-sudoku')

pprint.pprint(problem)

sudoku_board = problem['sudoku']

print('Input puzzle')
pretty_print(sudoku_string_to_matrix(sudoku_board))
print('\n')

# Solve the puzzle
solved_sudoku = min_conflicts(sudoku_board)
print('Solved puzzle')
pretty_print(solved_sudoku)
check_solution(sudoku_string_to_matrix(sudoku_board), solved_sudoku)

answer = {
    'sudoku': sudoku_matrix_to_string(solved_sudoku),
    'id': problem['id']
}

submit_answer('sudoku', answer)


{'id': 2,
 'sudoku': '100340597940817230367925814200408905098071362506290100801630700630750401750180620'}
Input puzzle
1 0 0 3 4 0 5 9 7
9 4 0 8 1 7 2 3 0
3 6 7 9 2 5 8 1 4
2 0 0 4 0 8 9 0 5
0 9 8 0 7 1 3 6 2
5 0 6 2 9 0 1 0 0
8 0 1 6 3 0 7 0 0
6 3 0 7 5 0 4 0 1
7 5 0 1 8 0 6 2 0


Solved puzzle
1 8 2 3 4 6 5 9 7
9 4 5 8 1 7 2 3 6
3 6 7 9 2 5 8 1 4
2 1 3 4 6 8 9 7 5
4 9 8 5 7 1 3 6 2
5 7 6 2 9 3 1 4 8
8 2 1 6 3 4 7 5 9
6 3 9 7 5 2 4 8 1
7 5 4 1 8 9 6 2 3
PASSED
